### TOOLS

In [1]:
import os
from langchain_groq import ChatGroq
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
model= ChatGroq(model="qwen/qwen3-32b")
response=model.invoke("Why do parrots fly?")
response.content

"<think>\nOkay, so the user is asking why parrots fly. Let me start by recalling what I know about parrots and their flying abilities. Parrots are birds, so flying is a natural behavior, but maybe there's more to it. I need to consider their biology, environment, and evolutionary reasons.\n\nFirst, flight in birds generally serves several purposes: escaping predators, finding food, migrating, and social interactions. For parrots specifically, they live in diverse habitats like rainforests, where dense canopies might require agile flying. So maybe they fly to navigate through trees or escape predators in those environments.\n\nAlso, parrots are known for their intelligence and social behavior. Do they use flight for social reasons, like staying with a flock or communicating? Maybe flight helps them maintain group cohesion. Additionally, some parrots migrate, but I think many are non-migratory. Wait, not all parrots migrate. So perhaps their flying is more about local movement within the

In [3]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It is sunny in {location}"

model_with_tools=model.bind_tools([get_weather])

response=model_with_tools.invoke("What is the temperature in Boston?")
for tool_call in response.tool_calls:
    print(f"Tool : {tool_call['name']}")
    print(f"Args:{tool_call['args']}")


Tool : get_weather
Args:{'location': 'Boston'}


### Tool Execution Loops

In [4]:
# Step 1: Model generates tool calls

messages=[{"role" : "user" , "content":"What is the weather in Boston?"}]
ai_msg= model_with_tools.invoke(messages)
messages.append(ai_msg)


#Step 2 Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    tool_result= get_weather.invoke(tool_call)
    messages.append(tool_result)


#Step 3 Pass results back to model for final response

final_response= model_with_tools.invoke(messages)
print(final_response.text)



The weather in Boston today is sunny.


In [5]:
messages

[{'role': 'user', 'content': 'What is the weather in Boston?'},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. I need to use the get_weather function. The function requires a location parameter, which in this case is Boston. So I\'ll call the function with "Boston" as the location. Let me make sure there are no typos. Everything looks good. I\'ll format the tool call as specified.\n', 'tool_calls': [{'id': 'rpgzy8b8n', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 153, 'total_tokens': 249, 'completion_time': 0.146690829, 'completion_tokens_details': {'reasoning_tokens': 72}, 'prompt_time': 0.006043502, 'prompt_tokens_details': None, 'queue_time': 0.158413998, 'total_time': 0.152734331}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_2bfcc54d36', 'service_tier': 'on_demand', 'finis